https://docs.databricks.com/en/notebooks/index.html

- set default language
- sql warehouse vs all purpose
- parameters: dbutils
- share result across sql and python
- visualize and add to dashboard
- install lib
- databricks assistant https://www.databricks.com/dataaisummit/session/maximizing-productivity-databricks-assistant-key-strategies-and-tips
- schedule notebook for recurring analysis 

Delta
- describe detail 
- transaction log
- time travel

Parameters (dbutils)

In [0]:
dbutils.widgets.text('age', '30')

In [0]:
age = dbutils.widgets.get('age')
print(age)

In [0]:
%sql
select :age

Share results

In [0]:
%sql
select
  1 + 1

In [0]:
new_dataframe_name = _sqldf 
display(new_dataframe_name)

## install lib
* leverage built-in lib ([dbr](https://docs.databricks.com/en/release-notes/runtime/16.1.html#system-environment), serverless version 1 vs 2)
* [notebook scope lib ](https://docs.databricks.com/en/libraries/notebooks-python-libraries.html)
  * When you install a notebook-scoped library, *only the current notebook and any jobs associated with that notebook have access to that library*. Other notebooks attached to the same cluster are not affected.
  * Notebook-scoped libraries do not persist across sessions. You must reinstall notebook-scoped libraries at the beginning of each session, or whenever the notebook is detached from a cluster.
* [cluster-scoped libraries](https://docs.databricks.com/en/libraries/cluster-libraries.html) : To install libraries for all notebooks attached to a cluster (non-serverless)
* [environments](https://docs.databricks.com/en/compute/serverless/dependencies.html#libraries) (serverless only) 

In [0]:
import simplejson

In [0]:
%pip install mlflow
# %pip install /path/to/my_package.whl
# %pip install -r /Workspace/shared/prod_requirements.txt

#%restart_python

Delta demo

In [0]:
%sql
use catalog joviloo;
create schema if not exists delta_basics;
use schema delta_basics;

In [0]:
%sql
CREATE TABLE delta_table (
  column1 INT,
  column2 STRING
);

INSERT INTO delta_table (column1, column2) VALUES
  (1, 'row1'),
  (2, 'row2');

In [0]:
%sql
describe detail delta_table
-- alternatively, look at uc directly 
-- format, s3 location, numFiles

In [0]:
%sql
INSERT INTO delta_table (column1, column2) VALUES
  (3, 'row3');

UPDATE delta_table
SET column2 = 'row2_new'
WHERE column1 = 2;

DELETE FROM delta_table where column1 = 1;

In [0]:
%sql
describe history delta_table;

In [0]:
%sql
select * from delta_table

In [0]:
%sql
select * from delta_table@v3 -- before delete operation on column1=1

In [0]:
%sql
RESTORE TABLE delta_table TO VERSION AS OF 3

In [0]:
%sql
select * from delta_table

udf
* pyspark udf (need to register using spark.udf.register, hence cannot be used with dbsql serverless)
* UC udf (can be used with dbsql serverless + all purpose serverless)

In [0]:
# can defined elsewhere and imported as function, then perform registration
def squared(s):
  return s * s
spark.udf.register("squaredWithPython", squared)

In [0]:
%sql
select squaredWithPython(3)

UC UDF

For UDFs that need to be accessible to all users, Databricks recommends creating a dedicated catalog and schema with appropriate access controls.

For team-specific UDFs, use a dedicated schema within the team’s catalog for storage and management.

Databricks recommends you include the following information in the UDF’s docstring:

The current version number

A changelog to track modifications across versions

The UDF’s purpose, parameters, and return value

An example of how to use the UDF

Here is an example of a UDF following best practices:

In [0]:
%sql
CREATE OR REPLACE FUNCTION joviloo.default.calculate_bmi(weight_kg DOUBLE, height_m DOUBLE)
RETURNS DOUBLE
COMMENT "Calculates Body Mass Index (BMI) from weight and height."
LANGUAGE PYTHON
AS $$
"""
Parameters:
calculate_bmi (version 1.2):
- weight_kg (float): Weight of the individual in kilograms.
- height_m (float): Height of the individual in meters.


Returns:
- float: The calculated BMI.


Example Usage:

SELECT calculate_bmi(weight, height) AS bmi FROM person_data;


Change Log:
- 1.0: Initial version.
- 1.1: Improved error handling for zero or negative height values.
- 1.2: Optimized calculation for performance.


 Note: BMI is calculated as weight in kilograms divided by the square of height in meters.
"""
if height_m <= 0:
  return None  # Avoid division by zero and ensure height is positive
return weight_kg / (height_m ** 2)
$$;


In [0]:
%sql
select joviloo.default.calculate_bmi(80,1.80)

databricks assistant